In [1]:
from pathlib import Path
import numpy as np
# import joblib
from keras.preprocessing import image
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam

In [2]:
#path to folders for training data
mask_path_train = Path("Face Mask Dataset") / "Train" / "WithMask"
no_mask_path_train = Path("Face Mask Dataset") / "Train" / "WithoutMask"

#path to folders for testing data
mask_path_test = Path("Face Mask Dataset") / "Validation" / "WithMask"
no_mask_path_test = Path("Face Mask Dataset") / "Validation" / "WithoutMask"

In [3]:
train_images = []
train_labels = []
test_images = []
test_labels = []

In [4]:
for img in no_mask_path_train.glob("*.png"):
    #Load the image from disk
    img = image.load_img(img)
    
    img = img.resize([128,128])
    
    #Convert the image to an array
    image_array = image.img_to_array(img)
    
    #Add the image to the list of images
    train_images.append(image_array)
    
    #For each "not mask image", the expected value should be 0
    train_labels.append(0)
    
for img in no_mask_path_test.glob("*.png"):
    #Load the image from disk
    img = image.load_img(img)
    
    img = img.resize([128,128])
    
    #Convert the image to an array
    image_array = image.img_to_array(img)
    
    #Add the image to the list of images
    test_images.append(image_array)
    
    #For each "not mask image", the expected value should be 0
    test_labels.append(0)

In [5]:
for img in mask_path_train.glob("*.png"):
    #Load the image from disk
    img = image.load_img(img)
    
    img = img.resize([128,128])
    
    #Convert the image to an array
    image_array = image.img_to_array(img)
    
    #Add the image to the list of images
    train_images.append(image_array)
    
    #For each "mask image", the expected value should be 1
    train_labels.append(1)
    
for img in mask_path_test.glob("*.png"):
    #Load the image from disk
    img = image.load_img(img)
    
    img = img.resize([128,128])
    
    #Convert the image to an array
    image_array = image.img_to_array(img)
    
    #Add the image to the list of images
    test_images.append(image_array)
    
    #For each "mask image", the expected value should be 1
    test_labels.append(1)

In [6]:
# Convert the array with train images we loaded into a numpy array
X_train = np.array(train_images)

# Convert the train lables array into a numpy array
Y_train = np.array(train_labels)

# Convert the array with test images we loaded into a numpy array
X_test= np.array(test_images)

# Convert the test lables array into a numpy array
Y_test = np.array(test_labels)

In [7]:
# Convert data from integer to floating point
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")

# Normalizing data between 0 and 1
X_train = X_train/255
X_test = X_test/255

#The labels are single labels : 0 or 1. I want them as an array with two values in whice one of them will be 1 ant the other, 0.
# Y_train = keras.utils.np_utils.to_categorical(Y_train, 2)
# Y_test = keras.utils.np_utils.to_categorical(Y_test, 2)

In [8]:
#Create a model and add layers
model = Sequential()

model.add(Conv2D(64, (3,3), padding="same", activation="relu", input_shape=(128,128,3)))
model.add(Conv2D(64, (3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128, (3,3), padding="same", activation="relu"))
model.add(Conv2D(128, (3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(256, (3,3), padding="same", activation="relu"))
model.add(Conv2D(256, (3,3), padding="same", activation="relu"))
model.add(Conv2D(256, (3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(512, (3,3), padding="same", activation="relu"))
model.add(Conv2D(512, (3,3), padding="same", activation="relu"))
model.add(Conv2D(512, (3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(512, (3,3), padding="same", activation="relu"))
model.add(Conv2D(512, (3,3), padding="same", activation="relu"))
model.add(Conv2D(512, (3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dropout(0.25))
model.add(Dense(units=4096,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(units=1, activation="sigmoid"))

opt = Adam(learning_rate=0.001)
# Compile the model
model.compile(
    loss=keras.losses.binary_crossentropy,
    optimizer = opt,
    metrics = ["accuracy"]
)


# Print a summary of the model
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 64)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 128)       73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 64, 64, 128)       147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 128)      0         
 2D)                                                    

In [9]:
# Train the model
model.fit(
    X_train,
    Y_train,
    batch_size=32,
    epochs = 20,
    validation_data = (X_test, Y_test),
    shuffle = True
)

# Save neural network structure
model_structure = model.to_json()
f = Path("VGG16_MASK_model_structure.json")
f.write_text(model_structure)

# Save neural network's trained weights
model.sample_weights("VGG16_MASK_model_weights.h5")

Epoch 1/20
 13/313 [>.............................] - ETA: 32:31 - loss: 0.7817 - accuracy: 0.5096

KeyboardInterrupt: 